In [ ]:
import os
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv
from PIL import Image, ImageDraw

def get_sp500_tickers():
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'id': 'constituents'})
        tickers = []
        for row in table.findAll('tr')[1:]:
            ticker = row.findAll('td')[0].text.strip()
            tickers.append(ticker)
        return tickers
    except Exception as e:
        print(f"Error fetching S&P 500 tickers: {e}")
        return []

def get_nasdaq100_tickers():
    try:
        url = "https://en.wikipedia.org/wiki/NASDAQ-100"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'id': 'constituents'})
        tickers = []
        for row in table.findAll('tr')[1:]:
            ticker = row.findAll('td')[1].text.strip()
            tickers.append(ticker)
        return tickers
    except Exception as e:
        print(f"Error fetching NASDAQ-100 tickers: {e}")
        return []

def get_nse_tickers():
    # Static list or alternative method, if scraping fails
    try:
        url = "https://www.moneycontrol.com/stocks/marketinfo/marketcap/nse/index.html"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'tbldata14 bdrtpg'})
        tickers = []
        for row in table.findAll('tr')[1:]:
            ticker = row.findAll('td')[0].text.strip().split(" ")[0]
            tickers.append(ticker)
        return tickers
    except Exception as e:
        print(f"Error fetching NSE tickers: {e}")
        return []

def calculate_rsi(df, window=14):
    delta = df['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    df['RSI'] = rsi
    return df

def find_rsi_signals(df):
    df['Buy_Signal'] = (df['RSI'] < 30) & (df['RSI'].shift(1) >= 30)
    df['Sell_Signal'] = (df['RSI'] > 70) & (df['RSI'].shift(1) <= 70)
    return df

def get_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end)
    return df

def plot_rsi_yearly(ticker, df, year, output_dir):
    df_year = df[df['Date'].dt.year == year]
    if df_year.empty:
        return None, None

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df_year['Date'], df_year['RSI'], label='RSI')
    ax.axhline(70, color='red', linestyle='--', label='Overbought (70)')
    ax.axhline(30, color='green', linestyle='--', label='Oversold (30)')

    buy_signals = df_year[df_year['Buy_Signal']]
    sell_signals = df_year[df_year['Sell_Signal']]
    bounding_boxes = []

    filename = f"{ticker}_{year}_RSI.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath)
    plt.close(fig)

    img = Image.open(filepath)
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size

    for i, row in buy_signals.iterrows():
        date_num = mdates.date2num(row['Date'])
        x, y = ax.transData.transform((date_num, row['RSI']))
        y = img_height - y
        box_width = 40
        box_height = 40
        xmin = int(x - box_width // 2)
        ymin = int(y - box_height // 2)
        xmax = int(x + box_width // 2)
        ymax = int(y + box_width // 2)
        bounding_boxes.append([xmin, ymin, xmax, ymax, "buy_signal"])

    for i, row in sell_signals.iterrows():
        date_num = mdates.date2num(row['Date'])
        x, y = ax.transData.transform((date_num, row['RSI']))
        y = img_height - y
        box_width = 40
        box_height = 40
        xmin = int(x - box_width // 2)
        ymin = int(y - box_height // 2)
        xmax = int(x + box_width // 2)
        ymax = int(y + box_width // 2)
        bounding_boxes.append([xmin, ymin, xmax, ymax, "sell_signal"])

    img.save(filepath)

    return bounding_boxes, filename

def visualize_bounding_boxes(image_dir, annotations_file, num_images=5):
    annotations = pd.read_csv(annotations_file)
    sample_images = annotations['image_filename'].unique()[:num_images]

    for image_filename in sample_images:
        image_path = os.path.join(image_dir, image_filename)
        img = Image.open(image_path)
        draw = ImageDraw.Draw(img)

        image_annotations = annotations[annotations['image_filename'] == image_filename]

        for _, row in image_annotations.iterrows():
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']
            label = row['label']
            color = "red" if label == "buy_signal" else "blue"

            draw.rectangle([xmin, ymin, xmax, ymax], outline=color)

        img.show()

# Combine all tickers
tickers = get_sp500_tickers() + get_nasdaq100_tickers() + get_nse_tickers()

if tickers:
    # Create output directory
    output_dir = 'rsi_images'
    os.makedirs(output_dir, exist_ok=True)

    annotations = []
    years = range(2000, 2020)  # Extend the range to nearly 20 years

    # Process each ticker
    try:
        for ticker in tickers:
            print(f"Processing ticker: {ticker}")
            df = get_data(ticker, start='2000-01-01', end='2019-12-31')
            if df.empty:
                print(f"No data for {ticker}. Skipping.")
                continue
            df['Date'] = pd.to_datetime(df.index)
            df = calculate_rsi(df)
            df = find_rsi_signals(df)

            for year in years:
                bounding_boxes, filename = plot_rsi_yearly(ticker, df, year, output_dir)
                if filename and bounding_boxes:
                    for bbox in bounding_boxes:
                        xmin, ymin, xmax, ymax, label = bbox
                        annotations.append([filename, xmin, ymin, xmax, ymax, label])

    except Exception as e:
        print(f"An error occurred during processing: {e}")

    # Save annotations to CSV
    try:
        annotations_file = 'rsi_annotations.csv'
        with open(annotations_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["image_filename", "xmin", "ymin", "xmax", "ymax", "label"])
            writer.writerows(annotations)
        print(f"Annotations saved to {annotations_file}.")
    except Exception as e:
        print(f"An error occurred while saving annotations: {e}")

    # Visualize bounding boxes
    visualize_bounding_boxes(output_dir, annotations_file)
else:
    print("No tickers to process.")
